##### **Evaluation Setup**:
* Pip Install Fabric Data Agent SDK
* Load the **DataFrame** with question and expected_answers list.
  * You can update in-cell DataFrame.
  * Or upload the CSV file in "question,expected_answer" format to lakehouse
    * Copy the file path and load the data to DataFrame using pandas.read_csv("<lakehouse_filepath>")
* Invoke the evaluate_data_agent API with data_frame, **data_agent_name**, workspace_name (Optional), table_name (Optional).
  * data_agent_name : Name of the Data Agent
  * workspace_name (Optional) : Workspace Name if Data Agent is in different workspace. Default value is None.
  * table_name (Optional) : Evaluation output table name to store the evaluation result. Default table name is 'evaluation_output'.
    * After evaluation there will be two tables one with provided <table_name> for evaluation output and other with <table_name>_steps for detailed steps.
  * data_agent_stage (Optional) : Data Agent stage i.e., sandbox or production. Default value is production.


#### Install Fabric Data Agent SDK

In [1]:
%pip install -U fabric-data-agent-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.8/351.8 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.6/442.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


##### Load the Dataframe using in cell initialization or input csv file

In [2]:
import pandas as pd

# Create DataFrame with "question,expected_answer". Please update the questions and expected_answers as per the requirement.
df = pd.DataFrame(columns=["question", "expected_answer"],
                  data=[
                    ["show total sales for Canadian Dollar for January 2013", "46,117.30."],
                    ["what is the product with the highest total sales for Canadian Dollar in 2013", "Mountain-200 Black, 42"],
                    ["Total sales outside of US", "19968887.95"],
                    ["which product category had the highest total sales for Canadian Dollar in 2013", "Bikes (Total Sales: 938654.76)"]
                ])

# Load from input CSV file with data in format "question,expected_answer"
# input_file_path = "/lakehouse/default/Files/Data/Input/curated_2.csv"
# df = pd.read_csv(input_file_path)


##### Invoke Evaluation API with input parameters

In [3]:
from fabric.dataagent.evaluation import evaluate_data_agent

# Data Agent name
data_agent_name = "AgentEvaluation"

# Workspace Name (Optional) if Data Agent is in different workspace
workspace_name = None

# Table name (Optional) to store the evaluation result. Default value is 'evaluation_output'
# After evaluation there will be two tables one with provided <table_name> for evaluation output and other with <table_name>_steps for detailed steps.
table_name = "demo_evaluation_output"

# Data Agent stage ie., sandbox or production. Default to production.
data_agent_stage = "production"

# Evaluate the Data Agent. Returns the unique id for the evaluation run
evaluation_id = evaluate_data_agent(df, data_agent_name, workspace_name=workspace_name, table_name=table_name, data_agent_stage=data_agent_stage)

Processing Rows: 100%|██████████| 4/4 [01:45<00:00, 26.28s/step]


##### Overall summary of an evaluation stored in the input table.
Returns the DataFrame with summary details.

Input Parameters:
* table_name (Optional) : Table name which contains the evaluation result. Default value is 'evaluation_output'
* verbose (Optional) : Flag to display the summary. Default is False.

In [4]:
from fabric.dataagent.evaluation import get_evaluation_summary

get_evaluation_summary(table_name)

index,evaluation_id,true_count,false_count,unclear_count,true_percentage
0,2cf897c6-801c-4e94-8664-f4848ffb6071,3,1,0,75.0
1,30c590da-6df8-494f-a359-642d34d4fa32,2,2,0,50.0
2,34fc1c4c-8696-48e5-aebb-78b3b681cc93,2,2,0,50.0
3,75d8e7db-3448-4e93-bb20-819e038e28a6,4,0,0,100.0
4,760af511-c8a8-497c-b729-6b113f4ac2a9,3,1,0,75.0
5,76e1e1c8-29ee-4505-bce2-9cd331927ef4,3,1,0,75.0
6,7b65fc7b-c8c8-46a1-9c2b-04adf65f4f48,3,1,0,75.0
7,934d185f-99da-4f48-908b-6a5a9c4fa9de,3,1,0,75.0
8,9e9c9c90-804c-4cb8-9831-ce80c36b53ac,3,1,0,75.0
9,ae71b8ee-8819-49a0-969a-42b0931ad202,3,1,0,75.0


##### Evaluation details of a single run
Returns the DataFrame with evaluation details.

Input Parameters:
* evaluation_id : Unique Id for an evaluation run.
* table_name (Optional) : Table name which contains the evaluation result. Default value is 'evaluation_output'.
* get_all_rows (Optional) : Flag to get all the rows for an evaluation. Default value is False, which returns only failed evaluation rows.
* Verbose (Optional) : Flag to display the summary. Default is False.

**Note**: The thread url in the evaluation details is only accessible by person who ran the evaluation.

In [5]:
from fabric.dataagent.evaluation import get_evaluation_details

# Unique Id for an evaluation run
evaluation_id = '9e9c9c90-804c-4cb8-9831-ce80c36b53ac'
# Evaluation output table name
table_name = "demo_evaluation_output"
# Flag to get all the rows for an evaluation. Default value is False, which returns only failed evaluation rows.
get_all_rows = False
# Flag to display the summary. Default is False.
verbose = True

eval_details = get_evaluation_details(evaluation_id, table_name, get_all_rows=get_all_rows, verbose=verbose)

question,expected_answer,evaluation_judgement,actual_answer,thread_url
Total sales outside of US,19968887.95,False,The total sales outside of the US are 1.00163 billion.,thread_TxIHbDOuCxzRfzdxZyqybg11


### Advanced Options

##### Use customized prompt for evaluation
* critic_prompt (Optional): Prompt (Optional) to evaluate the actual answer from Data Agent. 
  * Please use the variables **query, expected_answer and actual_answer** as placeholders.

In [6]:
from fabric.dataagent.evaluation import evaluate_data_agent

# Prompt (Optional) to evaluate the actual response. Please use the varaibles query, expected_answer and actual_answer as placeholders
critic_prompt = """
        Given the following query, expected answer, and actual answer, please determine if the actual answer is equivalent to expected answer. If they are equivalent, respond with 'yes'.

        Query: {query}

        Expected Answer:
        {expected_answer}

        Actual Answer:
        {actual_answer}

        Is the actual answer equivalent to the expected answer?
        """

# Data Agent name
data_agent_name = "AgentEvaluation"

# Evaluate the Data Agent. Returns the unique id for the evaluation run
evaluation_id = evaluate_data_agent(df, data_agent_name, critic_prompt=critic_prompt)

Processing Rows: 100%|██████████| 4/4 [01:46<00:00, 26.64s/step]


In [7]:
from fabric.dataagent.evaluation import get_evaluation_details

# Unique Id for an evaluation run
evaluation_id = '4e725e05-5b72-493f-b849-d8787decc188'
# Evaluation output table name
table_name = "evaluation_output"
# Flag to get all the rows for an evaluation. Default value is False, which returns only failed evaluation rows.
get_all_rows = True
# Flag to display the summary. Default is False.
verbose = True

eval_details = get_evaluation_details(evaluation_id, table_name, get_all_rows=get_all_rows, verbose=verbose)

question,expected_answer,evaluation_judgement,actual_answer,thread_url
show total sales for Canadian Dollar for January 2013,"46,117.30.",True,"The total sales for Canadian Dollar in January 2013 is 46,117.3.",thread_vpK7SupUSmaRbrxqBcZueJ7Q
what is the product with the highest total sales for Canadian Dollar in 2013,"Mountain-200 Black, 42",True,"The product with the highest total sales for Canadian Dollar in 2013 is ""Mountain-200 Black, 42"".",thread_ZOLweVJJQ4mrBa5pDnwYP6l8
Total sales outside of US,19968887.95,False,"The total sales outside of the US amount to approximately 1,001,630,000.The query used to obtain this information is:```sqlSELECT SUM(fs.SalesAmount) AS TotalSalesOutsideUSFROM FactInternetSales fsJOIN DimSalesTerritory dst ON fs.SalesTerritoryKey = dst.SalesTerritoryKeyJOIN DimGeography dg ON dst.SalesTerritoryCountry = dg.EnglishCountryRegionNameWHERE dg.EnglishCountryRegionName <> 'United States'```",thread_h4Hdx5Mt44L0gZrG9YKRco4p
which product category had the highest total sales for Canadian Dollar in 2013,Bikes (Total Sales: 938654.76),True,"The product category that had the highest total sales for Canadian Dollar in 2013 was ""Bikes"".",thread_NTMS5V17E96KR5FTtqkutJ7M
